A Hermitian system of size $n$

$$\mathbf y = \mathbf H^{-1}\mathbf v$$

can be embedded in a real, symmetric system of size $2n$:

\begin{equation}
\begin{bmatrix}
\Re(\mathbf y)\\\Im(\mathbf y)
\end{bmatrix}
=
\begin{bmatrix}
\Re(\mathbf H)&-\Im(\mathbf H)\\\Im(\mathbf H)&\Re(\mathbf H)
\end{bmatrix}^{-1}
\begin{bmatrix}
\Re(\mathbf v)\\\Im(\mathbf v)
\end{bmatrix}.
\end{equation}

Minimum-residual methods are often used for large problems, where constructing $H$ is impractical. In which case we may have an operation which computes a matrix-vector product, $f: \mathbb C^n \to \mathbb C^n; \,\, f(\mathbf v) = \mathbf H\mathbf v.$ This function can be wrapped to operate on $\mathbf x \in \mathbb R^{2n}$ by converting $\mathbf x$ back to a complex vector, applying $f$, and then embedding the result back in $\mathbb R^{2n}$.

Here is an example in python / numpy / scipy:

```python 
from scipy.sparse.linalg import minres, LinearOperator
from pylab import *

# Problem size
N = 100

# error helper
er = lambda t,a,b:print('%s error:'%t,mean(abs(a-b)))

# random Hermitian matrix
Q = randn(N,N) + 1j*randn(N,N)
H = Q@conj(Q.T)

# random complex vector
v = randn(N) + 1j*randn(N)

# ground-truth solution
x0 = inv(H)@v

# Pack/unpack complex vector as stacked real vector
c2r = lambda v:block([real(v),imag(v)])
r2c = lambda v:kron([1,1j],eye(N))@v

# Verify that we can embed C^n in R^(2N)
Hr = real(H)
Hi = imag(H)
Hs = block([[Hr,-Hi],[Hi,Hr]])
vs = c2r(v)
xs = inv(Hs)@vs
x1 = r2c(xs)
er('Embed',x0,x1)

# Verify that minres works as expected in R-embed
x2 = r2c(minres(Hs,vs,tol=1e-12)[0])
er('Minres 1',x0,x2)

# Demonstrate using operators
Av = lambda u:c2r( H @ r2c(u) )
A  = LinearOperator((N*2,)*2,Av,Av)

# Minres, converting input/output to/from complex/real
x3 = r2c(minres(Hs,vs,tol=1e-12)[0])
er('Minres 2',x0,x3)
```

```
>>> Embed error: 5.317184726020268e-12
>>> Minres 1 error: 6.641342200989796e-11
>>> Minres 2 error: 6.641342200989796e-11
```

In [3]:
from scipy.sparse.linalg import minres, LinearOperator
from pylab import *

# Problem size
N = 100

# error helper
er = lambda t,a,b:print('%s error:'%t,mean(abs(a-b)))

# random Hermitian matrix
Q = randn(N,N) + 1j*randn(N,N)
H = Q@conj(Q.T)

# random complex vector
v = randn(N) + 1j*randn(N)

# ground-truth solution
x0 = inv(H)@v

# Pack/unpack complex vector as stacked real vector
c2r = lambda v:block([real(v),imag(v)])
r2c = lambda v:kron([1,1j],eye(N))@v

# Verify that we can embed C^n in R^(2N)
Hr = real(H)
Hi = imag(H)
Hs = block([[Hr,-Hi],[Hi,Hr]])
vs = c2r(v)
xs = inv(Hs)@vs
x1 = r2c(xs)
er('Embed',x0,x1)

# Verify that minres works as expected in R-embed
x2 = r2c(minres(Hs,vs,tol=1e-12)[0])
er('Minres 1',x0,x2)

# Demonstrate using operators
Av = lambda u:c2r( H @ r2c(u) )
A  = LinearOperator((N*2,)*2,Av,Av)

# Minres, converting input/output to/from complex/real
x3 = r2c(minres(Hs,vs,tol=1e-12)[0])
er('Minres 2',x0,x3)

Embed error: 1.4302692278655435e-09
Minres 1 error: 1.5268243905585843e-08
Minres 2 error: 1.5268243905585843e-08
